## **1. Chunking & Embedding Strategies:**
* TT50, TT67 có cấu trúc (Thông tư > Điều > Khoản) & mỗi điều thường chứa 1 nội dung pháp lí hoàn chỉnh.
==> Vì vậy nên chunking theo cấu trúc tự nhiên, để maintain đủ context pháp lí và tránh cắt ngang văn bản giữa điều, khoản liên quan (chia theo điều - điều dài thì split, gắn metadata).
* Thêm overlap, thêm metadata (vì nếu phải chia điều nhỏ nếu điều quá dài so với "input accept amount" có thể bị lost context hoàn chỉnh của 1 điều):
  * *Metadata example: { "Thông tư": "tt67", "Điều": "4", "Khoản": "1" }*.
* Sử dụng các model embedding chuyên dụng cho Tiếng Việt:
  * *intfloat/multilingual-e5-large*, *multilingual-e5-base*, *vinai/phobert-base-v2*.

In [11]:
# 1.1 Input file => tt67, tt50
pdf_files = {
    'tt67': "tt-files/TT67.pdf",
    'tt50': "tt-files/TT50.pdf"
}

## **1. Import lib**

In [ ]:
%pip install pdfplumber sentence-transformers torch faiss-cpu

In [3]:
# 1. Import needed library
import pdfplumber # Best for Vietnamese
import re

from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import faiss

d:\cos30018\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **2. Extract & Preprocess PDF**

In [12]:
# BLOCK 2.1: Read in tt67, tt50
all_texts = {}
for tt_name, pdf_path in pdf_files.items():
  text = ""
  with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
       text += page.extract_text() or ""
  # Output: là 1 string chứa all 1 thông tư ==> (text)

  lines = text.split('\n')
  cleaned_lines = []
  for line in lines:
    line = line.strip()
    if line and not line.startswith('CỘNG HÒA') and not line.startswith('Độc lập') and not line.startswith('-------'):
      cleaned_lines.append(line)
  # Output: là 1 list với element là từng line trong 1 thông tư (cleaned_lines)


  cleaned_text = '\n'.join(cleaned_lines)
  all_texts[tt_name] = cleaned_text
  # Output: là 1 dict chứa content 2 thông tư > key: value là string dtype (all_texts)

In [13]:
# Kiểm tra block 2.1
for tt_name, text in all_texts.items():
  print(f"Sample cleaned text ({tt_name}):")
  print(text[:200])
  print(type(text))
  print()

Sample cleaned text (tt67):
BỘ TÀI CHÍNH CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Số: 67/2023/TT-BTC Hà Nội, ngày 02 tháng 11 năm 2023
THÔNG TƯ
HƯỚNG DẪN MỘT SỐ ĐIỀU CỦA LUẬT KINH DOANH BẢO HIỂM, NGHỊ ĐỊNH SỐ 46/2023/NĐ-
CP NGÀY 01 TH
<class 'str'>

Sample cleaned text (tt50):
BỘ TÀI CHÍNH CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Số: 50/2022/TT-BTC Hà Nội, ngày 11 tháng 8 năm 2022
THÔNG TƯ
HƯỚNG DẪN THỰC HIỆN MỘT SỐ ĐIỀU CỦA NGHỊ ĐỊNH SỐ 119/2015/NĐ-CP NGÀY 13 THÁNG 11 NĂM
2015 C
<class 'str'>



In [14]:
# BLOCK 3.1: Chunking theo điều
def chunk_by_article(text, tt_name):
  pattern = r'(Điều \d+\..*?)(?=Điều \d+\.|$)'  # Match từ "Điều X" đến trước "Điều X+1" hoặc hết văn bản
  chunks = re.findall(pattern, text, re.DOTALL)
  chunk_list = []
  metadata_list = []
  # Output: là 1 list với element là các điều của 1 thông tư, ví dụ thông tư 67 thì là 62 điều ==> (chunks)

  for i, chunk in enumerate(chunks):
    article_num = re.search(r'Điều (\d+)\.', chunk).group(1)
    metadata = {"Thông tư": tt_name, "Chương": "Unknown", "Điều": article_num}  # Thêm "Thông tư" vào metadata
    # Output: 1 dict metadata, aim là để ko mất context nếu có phải cắt đôi điều cho vừa input ==> (metadata)

    # Nếu chunk > 400 từ, chia nhỏ hơn (THAM SỐ NÀY NGỌC ANH vs PHƯƠNG CÓ THỂ MODIFY CHO PHÙ HỢP ĐẦU VÀO CỦA BỌN EM)
    if len(chunk.split()) > 400:
      sub_chunks = chunk.split('\n\n')
      for j, sub_chunk in enumerate(sub_chunks):
        if sub_chunk.strip():
          chunk_list.append(sub_chunk.strip())
          metadata_list.append({"Thông tư": tt_name, "Chương": "Unknown", "Điều": article_num, "Sub": j+1})
    else:
      chunk_list.append(chunk.strip())
      metadata_list.append(metadata)

  return chunk_list, metadata_list
  # Output: 2 list với len bằng nhau ==> (chunk_list - các điều và sub điều) & (metadata_list - metadata của các điều & sub điều tương ứng)

In [15]:
# BLOCK 3.2: APPLY
all_chunks = []
all_metadata = []
for tt_name, text in all_texts.items():
  chunks, metadata = chunk_by_article(text, tt_name)
  all_chunks.extend(chunks)
  # Output: chứa tất cả các chunk ==> (all_chunk)
  print(len(all_chunks))

  all_metadata.extend(metadata)
  # Ouput: chứa tát cả metadata của các chunk tương ứng ==> (all_metadata)
  print(len(all_chunks))

# ==> tt67 gồm 62 điều & tt50 gồm 37 điều, tức tổng là 99 chunks

62
62
99
99


In [16]:
# Kiểm tra block 3
for i in range(3):
  print(f"Chunk {i+1}: {all_chunks[i][:200]}... | Metadata: {all_metadata[i]}")
  print()

Chunk 1: Điều 1. Phạm vi điều chỉnh
1. Thông tư này quy định chi tiết khoản 3 Điều 14, khoản 2 Điều 17, khoản 4 Điều 76, khoản 4 Điều 82,
khoản 6 Điều 87, khoản 5 Điều 89, khoản 4 Điều 101, khoản 4 Điều 105, k... | Metadata: {'Thông tư': 'tt67', 'Chương': 'Unknown', 'Điều': '1'}

Chunk 2: Điều 2. Đối tượng áp dụng
1. Doanh nghiệp bảo hiểm phi nhân thọ, doanh nghiệp bảo hiểm nhân thọ, doanh nghiệp bảo hiểm sức
khỏe (sau đây gọi là doanh nghiệp bảo hiểm), doanh nghiệp tái bảo hiểm, đại l... | Metadata: {'Thông tư': 'tt67', 'Chương': 'Unknown', 'Điều': '2'}

Chunk 3: Điều 3. Cung cấp và cập nhật thông tin
1. Thông tin quy định tại điểm c khoản 1 Điều 7 Nghị định số 46/2023/NĐ-CP được quy định chi tiết
tại Mẫu số 1-CSDL Phụ lục I ban hành kèm theo Thông tư này.
2. ... | Metadata: {'Thông tư': 'tt67', 'Chương': 'Unknown', 'Điều': '3'}



## **4. Embedding**

In [17]:
# BLOCK 4: Embedding using Sentence Transformer
def get_vietnamese_embeddings(texts: list) -> np.ndarray:
  # Load mô hình Sentence Transformer tiếng Việt
  model = SentenceTransformer('keepitreal/vietnamese-sbert')

  embeddings = model.encode(
    texts,
    convert_to_numpy=True,
    show_progress_bar=False,
    device='cuda' if torch.cuda.is_available() else 'cpu'
  )
  # Output: embeddings là numpy array chứa vector embedding của tất cả văn bản, shape là (số văn bản, 768)

  return embeddings

In [18]:
embeddings_chunk = get_vietnamese_embeddings(all_chunks) # Test block 4, output là 1 np.array có dimension như bên dưới

In [19]:
embeddings_chunk.shape # 5 chunks với len 768

(99, 768)

## **5. Upload to Qdrant**

In [ ]:
%pip install dotenv qdrant-client

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta

In [ ]:
# BLOCK: Upload to Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import os
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv('.env.local')  # Load from .env.local

# Qdrant configuration
QDRANT_URL = os.getenv('QDRANT_URL')
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')
COLLECTION_NAME = 'legal_docs'

In [ ]:
def upload_to_qdrant(chunks, metadata_list, embeddings, batch_size=32):
    print(f"Connecting to Qdrant at {QDRANT_URL}")
    
    # Initialize Qdrant client
    client = QdrantClient(
        url=QDRANT_URL,
        api_key=QDRANT_API_KEY,
        prefer_grpc=False,
        timeout=60
    )
    
    # Test connection
    try:
        collections = client.get_collections()
        print(f"Successfully connected to Qdrant")
    except Exception as e:
        print(f"Connection failed: {str(e)}")
        print("Please check your Qdrant URL and API key")
        return False
    
    # Create collection if it doesn't exist
    embedding_size = embeddings.shape[1]
    if not client.collection_exists(COLLECTION_NAME):
        print(f"Creating collection '{COLLECTION_NAME}'...")
        client.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(
                size=embedding_size,
                distance=Distance.COSINE
            )
        )
        print(f"Collection created successfully")
    else:
        print(f"Collection '{COLLECTION_NAME}' already exists")
    
    # Upload data in batches
    total_batches = (len(chunks) + batch_size - 1) // batch_size
    start_time = time.time()
    
    for batch_idx in range(total_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(chunks))
        
        # Prepare batch of points
        points = []
        for i in range(start_idx, end_idx):
            points.append({
                "id": i + 1,  # Start IDs from 1
                "vector": embeddings[i].tolist(),
                "payload": {
                    "text": chunks[i],
                    "metadata": metadata_list[i]
                }
            })
        
        # Upload batch
        try:
            client.upsert(
                collection_name=COLLECTION_NAME,
                points=points
            )
            print(f"Batch {batch_idx + 1}/{total_batches} uploaded ({start_idx + 1}-{end_idx} of {len(chunks)})")
        except Exception as e:
            print(f"Failed to upload batch {batch_idx + 1}: {str(e)}")
            return False
    
    # Print upload statistics
    total_time = time.time() - start_time
    print(f"\nUpload completed successfully!")
    print(f"Total documents: {len(chunks)}")
    print(f"Total time: {total_time:.2f} seconds")
    
    return True

In [ ]:
# Run the upload
try:
    print("Starting upload to Qdrant...")
    success = upload_to_qdrant(all_chunks, all_metadata, embeddings_chunk)
    if success:
        print("Data successfully uploaded to Qdrant")
    else:
        print("Upload to Qdrant failed")
except Exception as e:
    print(f"Error during upload: {str(e)}")